In [1]:
import requests
import os
import json
import time
import pandas as pd
from dotenv import load_dotenv

In [9]:
api_key = "RGAPI-ed4b05bd-6a83-4ff8-963e-1e8cdf17bf30"
puuid = "7Se-dSzwQX7_yaK8T9Y9dBcnk6NvGrzTNIuDz5BXT-fsSKYh3wz0LlhCDGdklSVf9OivOyL2GMe60g"

In [ ]:
def getMatches(puuid, count):
    api_url = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/"
    api_url = api_url + puuid + "/ids?start=0&count=" + str(count)
    api_url = api_url + "&api_key=" + api_key

    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids

def getMatchData(match_id):
    api_url = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_id
    api_url = api_url + "?api_key=" + api_key
    resp = requests.get(api_url)
    match_data = resp.json()
    return match_data

def getMatchTimeline(match_id):
    api_url = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_id + "/timeline"
    api_url = api_url + "?api_key=" + api_key
    resp = requests.get(api_url)
    match_timeline = resp.json()
    return match_timeline

def getParticipants(match_data):
    participant_map = {}
    for p_info in match_data['info']['participants']:
        # Store champion name, and summoner name if available
        name_display = p_info['championName']
        if 'summonerName' in p_info and p_info['summonerName'] != "BOT": # Riot sometimes uses "BOT"
             name_display = f"{p_info['summonerName']} ({p_info['championName']})"

        participant_map[p_info['participantId']] = {
            "name": name_display,
            "champion": p_info['championName'],
            "teamId": p_info['teamId'] # 100 for blue, 200 for red
        }
    return participant_map

def format_timestamp_ms(milliseconds):
    """Converts milliseconds to a MM:SS string."""
    if milliseconds is None:
        return "N/A"
    total_seconds = milliseconds // 1000
    minutes = total_seconds // 60
    seconds = total_seconds % 60
    return f"{minutes:02d}:{seconds:02d}"

In [11]:
match_ids = getMatches(puuid, 100)
match_data = getMatchData(match_ids[0])
match_timeline = getMatchTimeline(match_ids[0])
participants = getParticipants(match_data)

In [ ]:
eventdictionary = {
    "event_type" : "CHAMPION_KILL",
    "victim" : "Rengar", 
    "start" : 0,
    "end" : 300
}

def eventWinrateStatistic(eventdictionary):
    wins = 0
    losses = 0
    # get match ids 
    match_ids = getMatches(puuid, 100)
    # for earch match id
    for match_id in match_ids:
        # get match data
        match_data = getMatchData(match_id)
        # get participant map
        participant_map = getParticipants(match_data)
        # get timeline
        match_timeline = getMatchTimeline(match_id)
        # find event in timeline
        for frame in match_timeline['info']['frames']:
            for event in frame['events']:
                if event['type'] == 'CHAMPION_KILL':
                    timestamp_ms = event['timestamp']
                    if eventdictionary['start']*1000 < timestamp_ms and timestamp_ms < eventdictionary['end']*1000:
                        victim_id = event.get('victimId') 
                        if participant_map[victim_id]['champion'] == eventdictionary['victim']:
                            
                    
                    #timestamp_ms = format_timestamp_ms(event['timestamp'])           

    return

{'status': {'message': 'Unknown apikey', 'status_code': 401}}